In [12]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

batch_size = 512
count = batch_size*100
epochs = 100

X = np.float32(np.random.uniform(-1, 1, size=(count, 3)))
y = np.zeros((count, 1), dtype=np.float32)
for i in range(count):
  y[i] = (X[i,0]*X[i,0] + X[i,1]*X[i,1] + X[i,2]*X[i,2]) ** 0.5 - 0.75

print(X)
print(y)

X = torch.from_numpy(X)
y = torch.from_numpy(y)
train_data = torch.utils.data.TensorDataset(X, y)

train_dataloader = DataLoader(train_data, batch_size=batch_size)

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class SinActivation(torch.nn.Module):
    def __init__(self):
        super(SinActivation, self).__init__()
        return
    def forward(self, x):
        return torch.sin(30.0*x)

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, layers, size):
        super().__init__()
        self.flatten = nn.Flatten()
        
        if (layers == 3):
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(3, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, 1)
          )
        elif (layers == 4):
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(3, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, 1)
          )
        elif (layers == 5):
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(3, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, 1)
          )
        elif (layers == 6):
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(3, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, size),
              SinActivation(),
              nn.Linear(size, 1)
          )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        #with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #if batch % 100 == 0:
        #    loss, current = loss.item(), (batch + 1) * len(X)
        #    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

[[ 0.11605557  0.15933192 -0.18424892]
 [-0.9897328  -0.46415344 -0.2459363 ]
 [ 0.46558908  0.7919273  -0.39744344]
 ...
 [-0.4856075   0.13017303  0.6853749 ]
 [-0.31313375  0.8677666  -0.9066895 ]
 [-0.4228776   0.9834448  -0.9688579 ]]
[[-0.48017928]
 [ 0.37048835]
 [ 0.2509412 ]
 ...
 [ 0.09999907]
 [ 0.5435059 ]
 [ 0.6938403 ]]
Using cuda device


In [13]:
import time

layers = [3, 4, 5]
sizes = [64, 128, 256]
epochs = 100

for t in range(5):
  for i in range(3):
    model = NeuralNetwork(layers[i], sizes[i]).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    t1 = time.time()
    for t in range(epochs):
      train(train_dataloader, model, loss_fn, optimizer)
    t2 = time.time()

    model.eval()
    
    t3 = time.time()
    with torch.no_grad():
      for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
    t4 = time.time()

    print("SIREN layers,size ", layers[i], sizes[i])
    print("training took", t2-t1)
    print("evaluation took", t4-t3)

SIREN layers,size  3 64
training took 37.57930874824524
evaluation took 0.2187199592590332


KeyboardInterrupt: 